In [2]:
#!
# region
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests, json, time
from scipy.stats import linregress
from citipy import citipy  # Incorporated citipy to determine city based on latitude and longitude
# Import API key
from config import weather_api_key
from iso3166 import countries

In [3]:
def k_to_c(temp):
    return temp - 273.15

In [4]:
def kmh_to_mph(speed):
    return speed * 0.621371

In [5]:
def north_or_south(latitude):
    return latitude < 0

In [6]:
api_key = weather_api_key
# endregion

Output File (CSV)

In [7]:
output_file = "output_data/cities.csv"

Range of latitudes and longitudes

In [8]:
lat_range = (-90, 90)
lng_range = (-180, 180)

In [9]:
lats = [np.random.uniform(lat_range[0], lat_range[1]) for i in range(3000)]
lngs = [np.random.uniform(lng_range[0], lng_range[1]) for i in range(3000)]

In [10]:
lat_lngs = list(zip(lats, lngs))

In [11]:
cities = []

In [12]:
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    city = city.strip()
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

In [13]:
len(cities)

979

In [15]:
temp_dict = {}

In [20]:
print(resp)
ol = []
i, j = 0, 0

{'coord': {'lon': -173.9833, 'lat': -18.65}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'base': 'stations', 'main': {'temp': 299.53, 'feels_like': 299.53, 'temp_min': 299.53, 'temp_max': 299.53, 'pressure': 1014, 'humidity': 61}, 'visibility': 10000, 'wind': {'speed': 4.63, 'deg': 100}, 'clouds': {'all': 20}, 'dt': 1635119179, 'sys': {'type': 1, 'id': 7292, 'country': 'TO', 'sunrise': 1635181157, 'sunset': 1635226820}, 'timezone': 46800, 'id': 4032420, 'name': 'Neiafu', 'cod': 200}


In [17]:
for city in cities:
    search_str = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'
    resp = requests.get(search_str)
    if resp.status_code == 200:
        if i < 500:
            resp = resp.json()
            vs = [
                    resp['coord']['lat'],
                    k_to_c(resp['main']['temp']),
                    resp['main']['humidity'],
                    kmh_to_mph(resp['wind']['speed']),
            ]
            i += 1
            temp_dict[city] = vs
        print(f'Request {j}: {city} request successful.')
    else:
        print(f'Request {j}: {city} request failed.')
    j += 1

Request 0: norman wells request successful.
Request 1: cidreira request successful.
Request 2: rikitea request successful.
Request 3: punta arenas request successful.
Request 4: attawapiskat request failed.
Request 5: east london request successful.
Request 6: talnakh request successful.
Request 7: hobart request successful.
Request 8: solnechnyy request successful.
Request 9: ushuaia request successful.
Request 10: upernavik request successful.
Request 11: arraial do cabo request successful.
Request 12: tuktoyaktuk request successful.
Request 13: ewa beach request successful.
Request 14: bethel request successful.
Request 15: charlottetown request successful.
Request 16: yellowknife request successful.
Request 17: marzuq request successful.
Request 18: turukhansk request successful.
Request 19: altensteig request successful.
Request 20: torbay request successful.
Request 21: broome request successful.
Request 22: sentyabrskiy request failed.
Request 23: faanui request successful.
Requ

KeyboardInterrupt: 